This notebook takes code from cropAll.py and uses it to crop the lat.rdr.full and lon.rdr.full files for geolocating an interferogram or velocity map of my cropped region with western mountains. <br><br>

Written 6-19-23. 

***

In [2]:
#!/usr/bin/env python3

import os
import glob
import re
from osgeo import gdal
import numpy as np

In [12]:
#%%script echo skipping
#import stackConfigs as sC


#-------------------------------------------------------
targetNumDates = 5 #may be able to increase this if you are using less than a full frame (<9-10 bursts, <3 swaths)
workdir  ='/data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600' #make sure this exists
#dem      = '/home/jovyan/LagunaSalada/DEM/demLat_N31_N33_Lon_W117_W115.dem.wgs84' #make sure this exists too!
#polygon = "&intersectsWith=POLYGON((-116.0339 31.7594,-115.1386 31.7594,-115.1386 32.7782,-116.0339 32.7782,-116.0339 31.7594))"
#track   = "&relativeOrbit=173" #also referred to as "relative orbit"
# now adding more dates to start on making a FULL stack 
#date1   = '&start=2017-05-10'  #start of search date period
#date2   = '&end=2022-01-10'    #end of search date period
#refdate = '20170510'
#pltdate = '20170510_20170522' #pair that you will plot in a later step   
rlooks     = 15 #range looks
alooks     = 4  #azimuth looks
looks      = ' -r '+str(rlooks)+' -z '+str(alooks)+' ' #final looks of your large-area interferograms
swaths     = ' -n \'1 2 3\''    #list of swaths if you know you only need one or two, otherwise all.
#this is the area you want to use to select bursts but may need be adjusted after your first iter of stackSentinel_VH
#narrowpoly = ' -b \'32.29 32.75 -116.0 -115.15\' ' 
# croppoly: min_range, min_azimuth, range_diff/width, azimuth_diff/width 
#croppoly   = np.array([[2558*15, 2872*4, 320*15, 200*4],[924*15, 3146*4, 420*15, 200*4]]) 
croppoly   = np.array([[14000, 1000, 4000, 1100]]) 
#-------------------------------------------------------

workDir=workdir

oldDirs = ['SLC_vv','SLC_vh','geom_reference']

#loop over polygons
for i in np.arange(croppoly.shape[0]): # if one croppoly, this only loops once 
    #the bounding box for this poly: upper left corner, width, length
    bb = croppoly[i,:] # this is the first and only croppoly.
    #name of new directory including crop info
    cropname = os.path.join(workDir,'cropped_'+str(bb[0])+'_'+str(bb[1])+'_'+str(bb[2])+'_'+str(bb[3]))
    print(cropname)
    for searchDir in oldDirs:
        testdir = os.path.join(workDir,searchDir)
        if os.path.isdir(testdir):
            if re.search(r'SLC',searchDir):
                for dateDir in os.listdir(testdir):
                    if re.search(r'\d{8}', dateDir):
                        oldDir=os.path.join(testdir,dateDir);
                        newDir=os.path.join(cropname,searchDir,dateDir);
                        os.makedirs(newDir, exist_ok=True)
                
                        for file in os.listdir(oldDir):
                            if file.endswith("slc.full"):
                                newfile=os.path.join(newDir,file)
                                if os.path.isfile(newfile):
                                    print(newfile+' already exists')
                                    
                                else:
                                    print('cropping '+os.path.join(oldDir,file))
                                
                                    ds = gdal.Open(os.path.join(oldDir,file))
                                    ds = gdal.Translate(newfile, ds, srcWin = bb,format='ISCE')
                                    ds = None
                    
            else:
                print('looking in '+testdir)
                oldDir=testdir
                newDir=os.path.join(cropname,searchDir);
                os.makedirs(newDir, exist_ok=True)
           
                for file in os.listdir(oldDir,):
                    if file.endswith("rdr.full") and re.match(r'\w{3}\.',file):
                        newfile=os.path.join(newDir,file)
                        if os.path.isfile(newfile):
                            print(newfile+' already exists')
                            
                        else:
                            print('cropping '+os.path.join(oldDir,file))
                            ds = gdal.Open(os.path.join(oldDir,file))
                            ds = gdal.Translate(newfile, ds, srcWin = bb,format='ISCE')
                            ds = None

/data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/cropped_14000_1000_4000_1100
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20211010/20211010.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20211115/20211115.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20180210/20180210.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20210218/20210218.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20200816/20200816.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20200810/20200810.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20211121/20211121.slc.full
cropping /data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/SLC_vv/20190406/20190406.slc.full
cropping /data/ocp4/Lagu

In [9]:
workDir  ='/data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600'
# croppoly: min_range, min_azimuth, range_diff/width, azimuth_diff/width 
bb = np.array([[14000, 1000, 4000, 1100]]) 
cropname = '/data/ocp4/LagunaSalada/Data_and_Figs/cropped_9000_1800_18000_3600/cropped_LS_with_western_mtns/geom_crop_test'
oldDir = ''

ds = gdal.Open(os.path.join(oldDir,file))
ds = gdal.Translate(newfile, ds, srcWin = bb,format='ISCE')
ds = None